In [33]:
pip install pyautogen

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


### Integration with local LLMs (From Hugging Face)

In [42]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="lmstudio-ai/gemma-2b-it-GGUF",
  messages=[
    {"role": "system", "content": "You are a well versed Doctor"},
    {"role": "user", "content": "How to have a healthy mind? Explain in 5 points"}
  ],
  temperature=0.7,
)

print(completion.choices[0].message.content)

1. **Exercise Regularly**: Regular physical activity not only boosts your heart health but also reduces stress and anxiety, which are major contributors to mental health issues.


2. **Eat Nutritiously**: A balanced diet rich in fruits, vegetables, whole grains, and lean proteins provides the body with essential nutrients for optimal brain function.


3. **Get Adequate Sleep**: Aim for 7-8 hours of sleep each night. During sleep, your brain consolidates memories and repairs tissues, promoting overall mental well-being.


4. **Manage Stress**: Find healthy ways to manage stress, such as meditation, yoga, or spending time in nature. Chronic stress can negatively impact mental health.


5. **Challenge Negative Thoughts**: Pay attention to negative self-talk and replace it with more positive and encouraging thoughts. Cultivating a positive mindset can lead to better emotional well-being and overall mental health.


### Setting up UserProxy agent & Assistant agent in a Groupchat

In [46]:
from autogen import ConversableAgent

gemma = {
    "config_list": [
        {
            "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

jack = ConversableAgent(
    "Jack (gemma)",
    llm_config=gemma,
    system_message="Your name is Jack and you are a fitness expert in two-person podcast.",
)


emma = ConversableAgent(
    "Emma (Gemma)",
    llm_config=gemma,
    system_message="Your name is Emma and you are a skincare expert in two-person podcast",
)

chat_result = jack.initiate_chat(emma, message="Emma, tell me a something about your expertise.", max_turns=2)

Jack (gemma) (to Emma (Gemma)):

Jack, tell me a something about your expertise.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (Gemma) (to Jack (gemma)):

Hey there! My name is Emma, and I'm a skincare expert with over 10 years of experience in the industry. I've worked with some of the biggest brands and clinics, and I'm passionate about helping people achieve their skin goals. I specialize in personalized skincare solutions, and I love guiding people to find products that work best for them.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jack (gemma) (to Emma (Gemma)):

That's awesome! It's great to hear that I can help people with their skincare goals. Are there any particular areas of skincare that you have expertise in?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (Gemma) (to 

In [53]:
import autogen

gemma = {
"config_list": [
    {
        "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
        "base_url": "http://localhost:1234/v1",
        "api_key": "lm-studio",
    },
],
"cache_seed": None,  # Disable caching.
}

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=gemma
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    }
)

user_proxy.initiate_chat(assistant, message="Write a python code for reversing 'word'")


user_proxy (to assistant):

Write a python code for reversing 'word'

--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
print("Enter a word: ")
word = input()

# Reverse the word using slicing and indexing
reversed_word = word[::-1]

# Print the reversed word
print("Reversed word:", reversed_word)
```
This code will take a word from the user and reverse it using slicing and indexing.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 2, in <module>
    word = input()
EOFError: EOF when reading a line


--------------------------------------------------------------------------------
assistant (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "Write a python code for reversing 'word'", 'role': 'assistant'}, {'content': '```python\nprint("Enter a word: ")\nword = input()\n\n# Reverse the word using slicing and indexing\nreversed_word = word[::-1]\n\n# Print the reversed word\nprint("Reversed word:", reversed_word)\n```\nThis code will take a word from the user and reverse it using slicing and indexing.', 'role': 'user'}, {'content': 'exitcode: 1 (execution failed)\nCode output: \nTraceback (most recent call last):\n  File "", line 2, in <module>\n    word = input()\nEOFError: EOF when reading a line\n', 'role': 'assistant'}, {'content': 'TERMINATE', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0, 'lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf': {'cost': 0, 'prompt_tokens': 1102, 'completion_tokens': 71, 'total_tokens': 1173}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gg

### Mapping external function calls to agent chat

In [70]:
import os
import autogen
from typing_extensions import Annotated
default_path = "backend_dir/"


gemma = {
    "config_list": [
        {
            "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
    }

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gemma,
    system_message="""
    I'm Engineer. I'm expert in python programming. I'm executing code tasks required by Admin.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER",
    code_execution_config=False,
)

groupchat = autogen.GroupChat(
    agents=[engineer, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemma)

@user_proxy.register_for_execution()
@engineer.register_for_llm(description="List files in choosen directory.")

def list_dir(directory: Annotated[str, "Directory to check."]):
    files = os.listdir(default_path + directory)
    return 0, files


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Check the contents of a chosen file.")
def see_file(filename: Annotated[str, "Name and path of file to check."]):
    with open(default_path + filename, "r") as file:
        lines = file.readlines()
    formatted_lines = [f"{i+1}:{line}" for i, line in enumerate(lines)]
    file_contents = "".join(formatted_lines)

    return 0, file_contents


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Replace old piece of code with new one. Proper indentation is important.")
def modify_code(
    filename: Annotated[str, "Name and path of file to change."],
    start_line: Annotated[int, "Start line number to replace with new code."],
    end_line: Annotated[int, "End line number to replace with new code."],
    new_code: Annotated[str, "New piece of code to replace old code with. Remember about providing indents."],
):
    with open(default_path + filename, "r+") as file:
        file_contents = file.readlines()
        file_contents[start_line - 1 : end_line] = [new_code + "\n"]
        file.seek(0)
        file.truncate()
        file.write("".join(file_contents))
    return 0, "Code modified"


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Create a new file with code.")
def create_file_with_code(
    filename: Annotated[str, "Name and path of file to create."], code: Annotated[str, "Code to write in the file."]
):
    with open(default_path + filename, "w") as file:
        file.write(code)
    return 0, "File created successfully"

The return type of the function 'list_dir' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'see_file' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'modify_code' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'create_file_with_code' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


In [71]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
You will need to improve app in FastApi. For now, check out all the application files, try to understand it and wait for next instructions.
""",
)

Admin (to chat_manager):


You will need to improve app in FastApi. For now, check out all the application files, try to understand it and wait for next instructions.


--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

I understand your role as an Engineer with expertise in Python programming. I am ready to assist you with understanding the application files and providing guidance based on my knowledge.

Please share the application files or any relevant information about the project, and I will do my best to help you improve the FastApi application.

--------------------------------------------------------------------------------

Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Engineer



BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}

### RetrieveAgents with task QA & code as RAG.

In [79]:
import json
import os

import chromadb

import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

import warnings
warnings.filterwarnings("ignore")

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

gemma = {
    "config_list": [
        {
            "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
    }


# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=gemma,
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": [
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
            os.path.join(os.path.abspath(""), "..", "website", "docs"),
        ],
        "custom_text_types": ["non-existent-type"],
        "chunk_token_size": 2000,
        "model": gemma["config_list"][0]['model'],
        # "client": chromadb.PersistentClient(path="/tmp/chromadb"),  # deprecated, use "vector_db" instead
        "vector_db": "chroma",  # to use the deprecated `client` parameter, set to None and uncomment the line above
        "overwrite": False,  # set to True if you want to overwrite an existing collection
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

In [78]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem, search_string="spark"
)  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2
Model lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['bdfbc921']]
Adding content of doc bdfbc921 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:

- Use Spark ML estimators for AutoML.
- Use Spark to run training in

### Group Chat with Retrieval Augmented Generation

In [54]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb
import autogen

gemma = {
    "config_list": [
        {
            "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1000,
        "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

coder = AssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message="You are a senior python engineer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=gemma,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=gemma,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=gemma,
)

PROBLEM = "How to use spark for parallel training in FLAML? Give me sample code."


def _reset_agents():
    boss.reset()
    boss_aid.reset()
    coder.reset()
    pm.reset()
    reviewer.reset()


def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, coder, pm, reviewer], messages=[], max_round=12, speaker_selection_method="round_robin"
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemma)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=3,
    )


def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer],
        messages=[],
        max_round=12,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemma)

    # Start chatting with the boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )


def call_rag_chat():
    _reset_agents()

    # In this case, we will have multiple user proxy agents and we don't initiate the chat
    # with RAG user proxy agent.
    # In order to use RAG user proxy agent, we need to wrap RAG agents in a function and call
    # it from other agents.
    def retrieve_content(message, n_results=3):
        boss_aid.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = boss_aid._check_update_context(message)
        if (update_context_case1 or update_context_case2) and boss_aid.update_context:
            boss_aid.problem = message if not hasattr(boss_aid, "problem") else boss_aid.problem
            _, ret_msg = boss_aid._generate_retrieve_user_reply(message)
        else:
            ret_msg = boss_aid.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message

    boss_aid.human_input_mode = "NEVER"  # Disable human input for boss_aid since it only retrieves content.

    llm_config = {
        "functions": [
            {
                "name": "retrieve_content",
                "description": "retrieve content for code generation and question answering.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
                        }
                    },
                    "required": ["message"],
                },
            },
        ],
        "config_list": gemma['config_list'],
        "timeout": 60,
        "cache_seed": 42,
    }

    for agent in [coder, pm, reviewer]:
        # update llm_config for assistant agents.
        agent.llm_config.update(llm_config)

    for agent in [boss, coder, pm, reviewer]:
        # register functions for all agents.
        agent.register_function(
            function_map={
                "retrieve_content": retrieve_content,
            }
        )

    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer],
        messages=[],
        max_round=12,
        speaker_selection_method="random",
        allow_repeat_speaker=False,
    )

    manager_llm_config = llm_config.copy()
    manager_llm_config.pop("functions")
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=manager_llm_config)

    # Start chatting with the boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )

In [56]:
norag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------

Next speaker: Senior_Python_Engineer

Senior_Python_Engineer (to chat_manager):

Sure, here's a sample code on how to use Spark for parallel training in FLAML:

```python
import flAML as fflm
from multiprocessing import Pool

# Load the data
data = fflm.load_data('path/to/data.h5')

# Define the model
model = fflm.create_model(data, 'my_model')

# Create a SparkSession
spark = fflm.spark_session()

# Create a Pool object to handle parallel tasks
pool = Pool(processes=4)

# Define the task function
def train_task(index):
    # Get the data for the current task
    data_subset = spark.read.load(f"data_{index}.h5")

    # Train the model with the data
    model.fit(data_subset)

# Submit the tasks to the pool
pool.map(train_task, range(10))

# Wait for all tasks to finish
pool.join()

# Stop the SparkSession
spark

BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}

In [59]:
rag_chat()

>Code_Reviewer
Boss_Assistant (to chat_manager):

Code_Reviewer

--------------------------------------------------------------------------------

Next speaker: Senior_Python_Engineer

Senior_Python_Engineer (to chat_manager):

Sure, here's the code you requested:

```python
# This is a comment

def main():
  """
  This function will print a message and then terminate the program.
  """
  print("Hello, world!")
  exit()


if __name__ == "__main__":
  main()
TERMINATE
```

When you run this code, it will print the message "Hello, world!" to the console and then exit the Python script immediately, returning control to the operating system.

I hope this is what you were looking for.

--------------------------------------------------------------------------------

Next speaker: Product_Manager

Product_Manager (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


In [60]:
call_rag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------

Next speaker: Product_Manager

Product_Manager (to chat_manager):

Sure! Here's an example of using Spark for parallel training in FLAML:

```python
from flamb import FLAML

# Load the data
data = load_data()

# Create a Spark dataframe from the loaded data
df = spark.createDataFrame(data)

# Define the model parameters
model_params = {
    "optimizer": "adam",
    "loss_function": "mse",
    "metrics": ["mae"],
}

# Create the FLAML model
model = FLAML(model_params, df)

# Train the model in parallel
model.fit(num_workers=4)

# Save the trained model
model.save("trained_model")

# Terminates the Spark session
TERMINATE
```

This code will:

1. Load the data into a Spark dataframe.
2. Create a Spark DataFrame from the loaded data.
3. Define the model parameters.
4. Create an FLAML model object using the paramet